## Perform single-cell quality control to remove erroneous, technical outlier segmentations

Using coSMicQC, we decided on three morphology features (related to shape and intensity) to maximize removal of poor quality nuclei segmentations that could impact downstream analysis.

The original indices are saved as a CSV file to be used for filtering downstream.

## Set papermill parameters

In [1]:
# Papermill notebook parameters
# (including notebook cell tag).
# We set a default here which may be overridden
# during Papermill execution.
# See here for more information:
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html
plate_id = "BR00117010"

In [2]:
# Parameters
plate_id = "BR00116999"


## Import libraries

In [3]:
from pathlib import Path

import cosmicqc
import pandas as pd
from cytodataframe import CytoDataFrame
from pyarrow import parquet

## Load in the plate data to process with only relevant metadata columns

In [4]:
# set path to save qc results
qc_results_dir = Path("./qc_results")
qc_results_dir.mkdir(parents=True, exist_ok=True)

# form a path to the parquet file with single-cell profiles
merged_single_cells = (
    f"/media/jenna/8TB-C/work/JUMP-single-cell/0.download_data/data/plates/{plate_id}/{plate_id}.parquet"
)

# read only the metadata from parquet file
parquet.ParquetFile(merged_single_cells).metadata

  created_by: parquet-cpp-arrow version 17.0.0
  num_columns: 7390
  num_rows: 390137
  num_row_groups: 44
  format_version: 2.6
  serialized_size: 48882510

In [5]:
# set a list of metadata columns for use throughout
metadata_cols = [
    "Metadata_ImageNumber",
    "Image_Metadata_Site",
    "Metadata_ObjectNumber",
    "Metadata_Plate",
    "Metadata_Well",
    "Image_Metadata_Col",
    "Image_Metadata_Row",
    "Image_FileName_CellOutlines",
    "Image_FileName_NucleiOutlines",
    "Image_FileName_OrigDNA",
    "Image_FileName_OrigRNA",
    "Nuclei_AreaShape_BoundingBoxMaximum_X",
    "Nuclei_AreaShape_BoundingBoxMaximum_Y",
    "Nuclei_AreaShape_BoundingBoxMinimum_X",
    "Nuclei_AreaShape_BoundingBoxMinimum_Y",
]

# read only metadata columns with feature columns used for outlier detection
df_merged_single_cells = pd.read_parquet(
    path=merged_single_cells,
    columns=[
        *metadata_cols,
        "Nuclei_AreaShape_FormFactor",
        "Nuclei_Intensity_MassDisplacement_DNA",
        "Nuclei_AreaShape_Compactness",
    ],
)
print(df_merged_single_cells.shape)
df_merged_single_cells.head()

(390137, 18)


,Metadata_ImageNumber,Image_Metadata_Site,Metadata_ObjectNumber,Metadata_Plate,Metadata_Well,Image_Metadata_Col,Image_Metadata_Row,Image_FileName_CellOutlines,Image_FileName_NucleiOutlines,Image_FileName_OrigDNA,Image_FileName_OrigRNA,Nuclei_AreaShape_BoundingBoxMaximum_X,Nuclei_AreaShape_BoundingBoxMaximum_Y,Nuclei_AreaShape_BoundingBoxMinimum_X,Nuclei_AreaShape_BoundingBoxMinimum_Y,Nuclei_AreaShape_FormFactor,Nuclei_Intensity_MassDisplacement_DNA,Nuclei_AreaShape_Compactness
0,1,1,1,BR00116999,A01,1,1,A01_s1--cell_outlines.png,A01_s1--nuclei_outlines.png,r01c01f01p01-ch5sk1fk1fl1.tiff,r01c01f01p01-ch3sk1fk1fl1.tiff,490,267,458,224,0.898131,0.209406,1.113423
1,1,1,2,BR00116999,A01,1,1,A01_s1--cell_outlines.png,A01_s1--nuclei_outlines.png,r01c01f01p01-ch5sk1fk1fl1.tiff,r01c01f01p01-ch3sk1fk1fl1.tiff,496,371,457,334,0.899824,0.269529,1.111328
2,2,2,1,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,652,39,615,10,0.903621,0.257325,1.106659
3,2,2,2,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,423,45,376,7,0.820085,1.069862,1.219385
4,2,2,3,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,593,45,571,21,0.922451,0.205589,1.084068


## Create mapping for the outlines to the images for CytoDataFrame

In [6]:
# create an outline and orig mapping dictionary to map original images to outlines
# note: we turn off formatting here to avoid the key-value pairing definintion
# from being reformatted by black, which is normally preferred.
# fmt: off
outline_to_orig_mapping = {
    rf"{record['Metadata_Well']}_s{record['Image_Metadata_Site']}--nuclei_outlines\.png":
    rf"r{record['Image_Metadata_Row']:02d}c{record['Image_Metadata_Col']:02d}f{record['Image_Metadata_Site']:02d}p(\d{{2}})-.*\.tiff"
    for record in df_merged_single_cells[
        [
            "Metadata_Well",
            "Image_Metadata_Row",
            "Image_Metadata_Col",
            "Image_Metadata_Site",
        ]
    ].to_dict(orient="records")
}
# fmt: on

next(iter(outline_to_orig_mapping.items()))

('A01_s1--nuclei_outlines\\.png', 'r01c01f01p(\\d{2})-.*\\.tiff')

## Detect technically mis-segmented nuclei

We define technically mis-segmented nuclei as segmentations that include more than one nuclei, under or over-segmentation, or segmentation of background/smudging.

We are using the measurements in two different conditions:


1. Irregular shaped nuclei where the intensity based center is much different than the shape's center

This condition looks to detect any technical outliers with irregular shape and very high difference in centeroids, which can be detecting multiple different types of technical outliers.

- `FormFactor`, which detects how irregular the shape is. 
- `MassDisplacement`, which detects how different the segmentation versus intensity based centeroids are (which can reflect multiple nuclei within one segmentation).
We understand that interesting phenotypes will occur, so we are going to evaluate if this will identify the mis-segmentationsb

### Detect outliers and show single-cell image crops with CytoDataFrame (cdf)

In [7]:
# find irregular shaped nuclei using thresholds that maximizes
# removing most technical outliers and minimizes good cells
feature_thresholds = {
    "Nuclei_AreaShape_FormFactor": -2.35,
    "Nuclei_Intensity_MassDisplacement_DNA": 1.5
}

irregular_nuclei_outliers = cosmicqc.find_outliers(
    df=df_merged_single_cells,
    metadata_columns=metadata_cols,
    feature_thresholds=feature_thresholds
)

irregular_nuclei_outliers_cdf = CytoDataFrame(
    data=pd.DataFrame(irregular_nuclei_outliers),
    data_context_dir=f"/media/jenna/8TB-C/work/JUMP-download-images/JUMP-single-cell/0.download_data/data/images/{plate_id}/orig",
    data_outline_context_dir=f"/media/jenna/8TB-C/work/JUMP-download-images/JUMP-single-cell/0.download_data/data/images/{plate_id}/outlines",
    segmentation_file_regex=outline_to_orig_mapping,
)[
    [
        "Nuclei_AreaShape_FormFactor",
        "Nuclei_Intensity_MassDisplacement_DNA",
        "Image_FileName_OrigDNA",
    ]
]


print(irregular_nuclei_outliers_cdf.shape)
irregular_nuclei_outliers_cdf.sort_values(by="Nuclei_AreaShape_FormFactor", ascending=False).head(2)

Number of outliers: 3073 (0.79%)
Outliers Range:
Nuclei_AreaShape_FormFactor Min: 0.2382392939636043
Nuclei_AreaShape_FormFactor Max: 0.6755315489397838
Nuclei_Intensity_MassDisplacement_DNA Min: 0.9385108762193086
Nuclei_Intensity_MassDisplacement_DNA Max: 7.352334038850726
(3073, 3)


,Nuclei_AreaShape_FormFactor,Nuclei_Intensity_MassDisplacement_DNA,Image_FileName_OrigDNA
225797,0.675532,0.959004,r08c10f02p01-ch5sk1fk1fl1.tiff
128039,0.675490,1.045053,r14c10f02p01-ch5sk1fk1fl1.tiff


### Randomly sample outlier rows to visually inspect if the threshold looks to be optimized

In [8]:
irregular_nuclei_outliers_cdf.sample(n=5, random_state=0)

,Nuclei_AreaShape_FormFactor,Nuclei_Intensity_MassDisplacement_DNA,Image_FileName_OrigDNA
226848,0.650406,1.964048,r11c03f05p01-ch5sk1fk1fl1.tiff
130641,0.642413,2.013209,r04c18f07p01-ch5sk1fk1fl1.tiff
109038,0.413985,2.200736,r10c05f05p01-ch5sk1fk1fl1.tiff
278593,0.486237,2.509497,r07c04f02p01-ch5sk1fk1fl1.tiff
283489,0.668746,1.439548,r16c12f04p01-ch5sk1fk1fl1.tiff


### Generate a new dataframe to save the original indices for filtering prior to further preprocessing

In [9]:
# Create a new dataframe for failed QC indices
failed_qc_indices = irregular_nuclei_outliers[metadata_cols].copy()
failed_qc_indices["original_index"] = failed_qc_indices.index  # Store original index
failed_qc_indices = failed_qc_indices.reset_index(drop=True)  # Reset index
failed_qc_indices["cqc.formfactor_displacement_outlier"] = True

print(failed_qc_indices.shape)
failed_qc_indices.head()

(3073, 17)


,Metadata_ImageNumber,Image_Metadata_Site,Metadata_ObjectNumber,Metadata_Plate,Metadata_Well,Image_Metadata_Col,Image_Metadata_Row,Image_FileName_CellOutlines,Image_FileName_NucleiOutlines,Image_FileName_OrigDNA,Image_FileName_OrigRNA,Nuclei_AreaShape_BoundingBoxMaximum_X,Nuclei_AreaShape_BoundingBoxMaximum_Y,Nuclei_AreaShape_BoundingBoxMinimum_X,Nuclei_AreaShape_BoundingBoxMinimum_Y,original_index,cqc.formfactor_displacement_outlier
0,183,3,2,BR00116999,A21,21,1,A21_s3--cell_outlines.png,A21_s3--nuclei_outlines.png,r01c21f03p01-ch5sk1fk1fl1.tiff,r01c21f03p01-ch3sk1fk1fl1.tiff,1062,459,1023,415,282,True
1,279,9,1,BR00116999,B07,7,2,B07_s9--cell_outlines.png,B07_s9--nuclei_outlines.png,r02c07f09p01-ch5sk1fk1fl1.tiff,r02c07f09p01-ch3sk1fk1fl1.tiff,426,90,372,57,448,True
2,353,2,1,BR00116999,B16,16,2,B16_s2--cell_outlines.png,B16_s2--nuclei_outlines.png,r02c16f02p01-ch5sk1fk1fl1.tiff,r02c16f02p01-ch3sk1fk1fl1.tiff,332,64,283,15,575,True
3,470,2,3,BR00116999,C05,5,3,C05_s2--cell_outlines.png,C05_s2--nuclei_outlines.png,r03c05f02p01-ch5sk1fk1fl1.tiff,r03c05f02p01-ch3sk1fk1fl1.tiff,281,65,227,13,765,True
4,592,7,3,BR00116999,C18,18,3,C18_s7--cell_outlines.png,C18_s7--nuclei_outlines.png,r03c18f07p01-ch5sk1fk1fl1.tiff,r03c18f07p01-ch3sk1fk1fl1.tiff,348,99,275,67,974,True


2. Nuclei segmentations with holes and also irregular shaped

We need to include this extra condition as it was discovered that there were more poorly segmented cells not caught, especially those over-segmented nuclei that contained holes, which is not common for a nuclei segmentation.

- `Compactness`, which detects irregular nuclei and nuclei containing holes.

### Detect outliers and show single-cell image crops with CytoDataFrame (cdf)

In [10]:
# find mis-segmented nuclei due using thresholds that maximizes
# removing most technical outliers and minimizes good cells
feature_thresholds = {
    "Nuclei_AreaShape_Compactness": 4.05,
}

poor_nuclei_outliers = cosmicqc.find_outliers(
    df=df_merged_single_cells,
    metadata_columns=metadata_cols,
    feature_thresholds=feature_thresholds
)

poor_nuclei_outliers_cdf = CytoDataFrame(
    data=pd.DataFrame(poor_nuclei_outliers),
    data_context_dir=f"/media/jenna/8TB-C/work/JUMP-download-images/JUMP-single-cell/0.download_data/data/images/{plate_id}/orig",
    data_outline_context_dir=f"/media/jenna/8TB-C/work/JUMP-download-images/JUMP-single-cell/0.download_data/data/images/{plate_id}/outlines",
    segmentation_file_regex=outline_to_orig_mapping,
)[
    [
        "Nuclei_AreaShape_Compactness",
        "Image_FileName_OrigDNA",
    ]
]


print(poor_nuclei_outliers_cdf.shape)
poor_nuclei_outliers_cdf.sort_values(by="Nuclei_AreaShape_Compactness", ascending=True).head(2)

Number of outliers: 3115 (0.80%)
Outliers Range:
Nuclei_AreaShape_Compactness Min: 1.7064852245580984
Nuclei_AreaShape_Compactness Max: 5.107239178423518
(3115, 2)


,Nuclei_AreaShape_Compactness,Image_FileName_OrigDNA
351386,1.706485,r03c23f02p01-ch5sk1fk1fl1.tiff
361590,1.706535,r07c04f08p01-ch5sk1fk1fl1.tiff


### Randomly sample outlier rows to visually inspect if the threshold looks to be optimized

In [11]:
poor_nuclei_outliers_cdf.sample(n=5, random_state=0)

,Nuclei_AreaShape_Compactness,Image_FileName_OrigDNA
296262,2.407886,r10c05f02p01-ch5sk1fk1fl1.tiff
336848,1.782967,r07c19f03p01-ch5sk1fk1fl1.tiff
369392,1.708414,r06c09f02p01-ch5sk1fk1fl1.tiff
185977,1.910645,r14c10f07p01-ch5sk1fk1fl1.tiff
332592,1.863542,r15c24f03p01-ch5sk1fk1fl1.tiff


### Save the original indices for failed single cells to compressed CSV files

In [12]:
# Create a new dataframe for poor nuclei outliers
poor_qc_indices = poor_nuclei_outliers[metadata_cols].copy()
poor_qc_indices["original_index"] = poor_qc_indices.index  # Store original index
poor_qc_indices = poor_qc_indices.reset_index(drop=True)  # Reset index
poor_qc_indices["cqc.compactness_outlier"] = True

# Merge both outlier dataframes on metadata columns and original_index
failed_qc_indices = failed_qc_indices.merge(
    poor_qc_indices,
    on=[*metadata_cols, "original_index"],
    how="outer"
)

# Fill missing values with False (ensuring only detected outliers are True)
failed_qc_indices[["cqc.formfactor_displacement_outlier", "cqc.compactness_outlier"]] = (
    failed_qc_indices[["cqc.formfactor_displacement_outlier", "cqc.compactness_outlier"]]
    .fillna(False)
)

# Calculate percentage removed
num_outliers = failed_qc_indices["original_index"].nunique()
total_cells = len(df_merged_single_cells)
percentage_removed = (num_outliers / total_cells) * 100

# Save the indices for failed single-cells as a compressed CSV file
failed_qc_indices.to_csv(Path(f"{qc_results_dir}/{plate_id}_failed_qc_indices.csv.gz"), compression="gzip")  # noqa: E501

print(f"Failed QC indices shape: {failed_qc_indices.shape}")
print(f"Percentage of single cells removed: {percentage_removed:.2f}%")
failed_qc_indices.head()

Failed QC indices shape: (5084, 18)
Percentage of single cells removed: 1.30%


/tmp/ipykernel_1942562/2551156104.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  failed_qc_indices[["cqc.formfactor_displacement_outlier", "cqc.compactness_outlier"]]


,Metadata_ImageNumber,Image_Metadata_Site,Metadata_ObjectNumber,Metadata_Plate,Metadata_Well,Image_Metadata_Col,Image_Metadata_Row,Image_FileName_CellOutlines,Image_FileName_NucleiOutlines,Image_FileName_OrigDNA,Image_FileName_OrigRNA,Nuclei_AreaShape_BoundingBoxMaximum_X,Nuclei_AreaShape_BoundingBoxMaximum_Y,Nuclei_AreaShape_BoundingBoxMinimum_X,Nuclei_AreaShape_BoundingBoxMinimum_Y,original_index,cqc.formfactor_displacement_outlier,cqc.compactness_outlier
0,2,2,17,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,558,366,497,300,49723,True,True
1,2,2,18,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,921,441,867,382,49724,False,True
2,2,2,20,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,713,500,678,414,59240,False,True
3,2,2,38,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,963,610,925,552,113080,False,True
4,2,2,39,BR00116999,A01,1,1,A01_s2--cell_outlines.png,A01_s2--nuclei_outlines.png,r01c01f02p01-ch5sk1fk1fl1.tiff,r01c01f02p01-ch3sk1fk1fl1.tiff,762,608,707,576,114264,False,True
